# Training Model: SVM

## 1. Importing libraries ...

In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.ml_prep import prep_ml
from jwlab.ml_prep_multigroup import prep_ml_multigroup
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

## 2. Importing Participants..

In [2]:

#participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
#participants = [ "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
#9m with >40 trials
#participants = [ "909", "912", "908", "913", "914", "916", "917", "919", "920", "921", "924","927", "930"]

#12m all
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]
#12m with >40 trials
#participants = ["109", "111", "112", "115", "124"]

#all participants
#participants = [ "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932",
#               "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]



## 3. Set up averaging

In [3]:
X, y, good_trial_count = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")


loaded
loaded
loaded


## 3. b) Different groups

In [3]:
X, y, good_trial_count = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")



loaded
loaded
loaded


In [4]:
#good_trial_count

In [5]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]


In [6]:
n

684

In [4]:
unique, counts = np.unique(y[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 19, 1.0: 16}

In [6]:
(n, d) = Xt[0][0].shape
assert n == yt[0][0].shape[0]


n

208

In [7]:
d

60000

In [8]:
unique, counts = np.unique(yt[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 104, 1.0: 104}

In [9]:
(n, d) = Xa[0][0].shape
assert n == ya[0][0].shape[0]
n

16

In [10]:
d

60000

In [11]:
unique, counts = np.unique(ya[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 8, 1.0: 8}

## 4. Setting up the SVM model ...

In [4]:
#model = LinearSVC(C=1e-9, max_iter=5000)
model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

## 5. Training and testing the model ...

### 5.1. Train on raw, test on raw (validation)

In [5]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(X[0][0]) != y[0][0])

0.6374269005847953

### 5.2. Train on raw, test on avg by trial (word repetition) 

In [6]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(Xt[0][0]) != yt[0][0])

0.625

### 5.3. Train on raw, test on avg by word and ps 

In [7]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(Xa[0][0]) != ya[0][0])

0.625

# 6 Subset analysis 

## 6.1 Generating random subsets of the chosen participant list

In [3]:
participants_train, participants_test = train_test_split(participants,test_size=0.2)
print(len(participants_train), len(participants_test))

10 3


### 6.2 Create train and test sets: animates/inanimates

In [4]:
X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

loaded
loaded


### 6.2 Create train and test sets - multi group

In [4]:
#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

loaded
loaded


## 6.3 Radial Classification

In [5]:
model = SVC(gamma=.001, kernel = 'rbf', C = 100)

model.fit(X_train[0][0], y_train[0][0])
np.mean(model.predict(X_test[0][0]) != y_test[0][0])

0.6875

## 6.4 Alternate accuracy measurements

In [6]:
from sklearn.model_selection import train_test_split

y_pred = model.predict(X_test[0][0])

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test[0][0],y_pred)))
print('Precision Score : ' + str(precision_score(y_test[0][0],y_pred)))
print('Recall Score : ' + str(recall_score(y_test[0][0],y_pred)))
print('F1 Score : ' + str(f1_score(y_test[0][0],y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test[0][0],y_pred)))

Accuracy Score : 0.34328358208955223


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

### 7 Optimization

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}
svc = SVC()
model = GridSearchCV(svc, parameters, verbose=True)
model.fit(X_train[0][0], y_train[0][0])

model.cv_results_

/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.4s finished
/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.02266153, 0.01959101, 0.02000801, 0.01917036, 0.01376764,
        0.01701641, 0.01506201, 0.01267608, 0.01388582, 0.01280793,
        0.01223063, 0.0232319 , 0.01872547, 0.01688528, 0.01579467,
        0.02559988, 0.0155433 , 0.01337814, 0.01234516, 0.01175992]),
 'std_fit_time': array([0.01289519, 0.00712717, 0.00498096, 0.00097637, 0.00193705,
        0.0016657 , 0.00226809, 0.00216544, 0.00279983, 0.00180435,
        0.00174231, 0.01558891, 0.00515244, 0.00585967, 0.00120147,
        0.00435602, 0.00189483, 0.00191716, 0.00144484, 0.0014357 ]),
 'mean_score_time': array([0.0047098 , 0.00611409, 0.01631705, 0.01318908, 0.00439707,
        0.00480326, 0.00517058, 0.00548808, 0.0052859 , 0.00437117,
        0.00454672, 0.00746441, 0.00547059, 0.00863202, 0.00574231,
        0.00755858, 0.00460561, 0.00432905, 0.00405637, 0.00400424]),
 'std_score_time': array([0.00019946, 0.00194631, 0.01467569, 0.00351266, 0.00041806,
        0.0003233 , 0.00044929, 0.000501

In [6]:
model.best_score_

0.5

In [7]:
model.best_estimator_

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Cross Validation (5 fold)

In [4]:

#model = LinearSVC(max_iter=5000, C=1e-9)
model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)


In [5]:
scores = cross_val_score(model, X[0][0], y[0][0], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.50364964 0.50364964 0.50364964 0.50364964 0.5       ]
Accuracy: 0.50 (+/- 0.00)


In [6]:
scores = cross_val_score(model, Xt[0][0], yt[0][0], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.38095238 0.5        0.35714286 0.47619048 0.425     ]
Accuracy: 0.43 (+/- 0.11)


In [7]:
scores = cross_val_score(model, Xa[0][0], ya[0][0], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.5 0.5 0.5 0.  0.5]
Accuracy: 0.40 (+/- 0.40)
